# **Trabalho Avaliativo 2 - Inteligência Artificial**
**Acadêmicos:**
- Guilherme Farias Stefani
- Henrique Baptista de Oliveira
- Henrique Cardoso Zanette
- Jhonata Saraiva Peres
- Lucas Dellatorre de Freitas


___

### **Bibliotecas Python nescessarias para utilização deste notebook:**

- **Pandas:** Para leitura do DataSet

- **Unidecode:** Para normalização dos textos

- **Regex:** Para normalização dos textos

- **Spacy:** Para lematização

- **Ploty:** Para gráficos e afins

- **Scikit Learn:** Para utilizar métodos de normalização e algoritmos de machine learning


*Rodar no powershell como administrador*

> - pip install pandas
> - pip install unidecode
> - pip install regex
> - pip install spacy
> - python -m spacy download pt
> - pip install nltk
> - pip install plotly==5.11.0
> - pip install -U scikit-learn

*Import das bibliotecas*

In [733]:
import pandas as pd
import regex as reg
import spacy
import unidecode
import plotly.express as px
from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download('rslp')

stemmer = nltk.stem.RSLPStemmer()



[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\guilh\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


___

### **Repositório de arquivos, datasets e dados de normalização**

*Localização dos arquivos de resources*

In [734]:
filenameDataSet = 'resources/DataSet.xlsx'
filenameRegexToBeApplied =  'resources/regexToBeApplied'
filenameStopWords = 'resources/stopWords'
fileNameDeParas = 'resources/dePara'

___

### **Dataset e amostras**

*Amostra completa do dataset*

In [735]:
dataSet = pd.ExcelFile(filenameDataSet)

In [736]:
dataSetRaw = pd.read_excel(dataSet, sheet_name='DebateGovRS')

*Amostra do dataset apenas com retweets*

In [737]:
retweets = pd.read_excel(dataSet, sheet_name='RT')

*Amostra do dataset classificado/rotulado*

In [738]:
rotulados = pd.read_excel(dataSet, sheet_name='ROTULADO')

fig = px.bar(rotulados['text'], x=rotulados['rotulo'])
fig.show()

*Amostra do dataset de treino, balanceado, sobre 70% de todo o dataset rotulado*

In [739]:
treino = pd.read_excel(dataSet, sheet_name='TREINO_70%')

fig = px.bar(treino['text'], x=treino['rotulo'])
fig.show()

*Amostra do dataset de teste, com 15% sobre os 30% restantes do dataset rotulado não utilizado pela amostra de treino*

In [740]:
teste = pd.read_excel(dataSet, sheet_name='TESTE_15%')

fig = px.bar(teste['text'], x=teste['rotulo'])
fig.show()

___

### **Classificação e Limpeza dos Dados**

- Inicialmente, retiramos todos os retweets presentes no dataset;

- Em seguida, definimos os tipos de classificação dos dados, conforme abaixo:
>   - **Contexto:** Precisa de contexto para ser entendido
>   - **Positivo:**  Mensagens positivas: Elogios, enaltecimento de candidato, etc.
>   - **Negativo:** Mensagens negativas: Ofensas, xingamentos, desmerecer propostas.
>   - **Ironia:** Sentido contrario da mensagem
>   - **Neutro:** Nem negativo nem positivo. Somente descritivo (narração de fatos)
>   - **Confuso:** Não foi possível interpretar o significado do texto
>   - **Dividir:** Mensagens positivas e negativas com possibilidade de quebra para geração de novos textos

- Após a definição dos tipos de classificação, começamos a classificar individualmente e manualmente, um por um;

- Depois da classificação individual, realizamos uma revisão geral em grupo, onde foram ponderadas e debatidas as classificações, também de forma manual, percorrendo por todas as linhas do dataset;

___

### **Normalização dos dados**

Para normalizar os dados, utilizamos de algumas ferramentas e técnicas a fim de garantir um melhor resultado pelo algoritmo, assim realizamos os seguintes passos:


*Inicialmente, todo o input foi transformado para caixa baixa, as quebras de linha foram removidas do texto e os espaços duplos foram transformados em espaço único*

In [741]:
def toLower(input):
    return input.lower()

def removeLinesBreaks(input):
    return reg.sub('\\\\n{1,}|\\\\r{1,}', ' ', input, reg.IGNORECASE, reg.MULTILINE)

def removeDoubleSpaces(input):
    return reg.sub('\\s{2,}', ' ', input, reg.IGNORECASE, reg.MULTILINE)

def applyRegex(input):
    regexs = open(filenameRegexToBeApplied, 'r', encoding='UTF-8', newline='\n')
    for regex in regexs.readlines():  
        teste = reg.sub(regex, ' ', input, reg.IGNORECASE, reg.MULTILINE)
        input = teste
    return input

In [742]:
def applyStemmer(input):

    input = reg.sub('\\s{2,}', ' ', input, reg.IGNORECASE, reg.MULTILINE).split(' ')
    input = filter(lambda x: x != ' ' and x != '', input)

    palavras = [stemmer.stem(palavra) for palavra in input]

    input = ''

    for palavra in palavras:
        input += palavra + ' '

    return reg.sub('\\s{2,}', ' ', input, reg.IGNORECASE, reg.MULTILINE)


*Em seguida, utilizando um arquivo customizado desenvolvido pelo grupo, transformamos palavras e nomes com diversas variações para uma forma única, como por exemplo, Você ou vc para voce*

In [743]:
def dePara(input):
    deParas = open(fileNameDeParas, 'r', encoding='UTF-8', newline='\n')
    for dePara in deParas.readlines():
        dePara = dePara.split('(-*-)')
        input = reg.sub(dePara[0], dePara[1], input, reg.IGNORECASE, reg.MULTILINE)
    return input

*Após a transformação das palavras, utilizamos a técnica de stop words para remover as palavras que não trazem significado para o texto, como por exemplo, os artigos: a, as, o, os*

In [744]:
def removeStopWords(input):
    stopWrods = open(filenameStopWords, 'r', encoding='UTF-8', newline='\r\n')
    for stopWord in stopWrods.readlines():
        stopWord = unidecode.unidecode(stopWord)
        stopWord = reg.sub('\r\n', '', stopWord)
        input = reg.sub(stopWord, '', input, reg.IGNORECASE, reg.MULTILINE)
    return input

*Ainda no processo de normalização, removemos todos os acentos das palavras do dataset, a fim de padronizar os dados para posterior análise pelo algoritmo*

In [745]:
def removeAcentuacao(input):
    return unidecode.unidecode(input)

Por fim, juntamos as etapas de normalização em uma unica função. Assim podemos utiliza-la de maneira mais fácil.

In [746]:
def normaliza(dataFrame):
    listAux = []
    for line in dataFrame.values:
        line[1] = removeDoubleSpaces(applyStemmer(dePara(removeLinesBreaks(removeAcentuacao(removeStopWords(toLower(unidecode.unidecode(line[1]))))))))
        listAux.append(line)
    return pd.DataFrame(listAux)

In [747]:
rotulados = normaliza(rotulados)

___

### **Criação dos conjuntos de treino, teste e validação**

- A partir de todo o conjunto de dados classificado, foram mantidos apenas os valores positivos e negativos (Ironias e confusões foram removidas e divisões possíveis não foram realizadas)

- Ordenamos por ordem alfabetica para retirar os textos duplicados do conjunto de dados

- Utilizando o tipo de dado com menor amostras (POSITIVO), foi determinado 70% de suas linhas, a quantidade necessária para o conjunto de treino.

- Antes de obter os dados e alocar-los no conjunto de treino, sua ordenação foi aleatorizada para abrangir uma quantidade maior de textos.

- Após aleatorizar foram colocados no conjunto de treino, com quantidades iguais de positivos e negativos.

- Posteriormente foram criados os conjuntos de testes e validação, ambos com 15% cada, do total de amostras do tipo POSITIVO (com menor amostras), com os valores também ordenados de forma aleatória, mas balanceados entre positivos e negativos

___

### **Bag of Words**

 Para realizar a bag of words precisamos converter o texto para um formato que a máquina consiga entender, pois ela não compreende texto nem caracteres. Para isso, utilizamos o método CountVectorizer, cuja finalidade é converter as palavras para o número de sua frequência. Dessa forma, se tivermos o seguinte texto:

 - text = [‘Oi, meu nome é lucas, esse é meu python notebook’]
 
 O CountVectorizer vai converter para:

 - Oi: 1, é: 2, lucas: 1, esse: 1, meu: 2, python: 1, notebook: 1





In [748]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

tfi = TfidfTransformer(use_idf=True)
vectorizer = CountVectorizer(analyzer = "word")
normaliza(treino)

treino_tweets = treino["text"].values
treino_classificacao = treino["rotulo"].values

vec_treino = vectorizer.fit_transform(treino_tweets)
freq_treino = tfi.fit_transform(vec_treino)

Bag of Words dataset de validacação

In [749]:
validacao = pd.read_excel(dataSet, sheet_name='VALIDACAO_15%')
normaliza(validacao)

validacao_tweets = validacao["text"].values
validacao_classificacao = validacao["rotulo"].values

vec_validacao = vectorizer.transform(validacao_tweets)
freq_validacao = tfi.transform(vec_validacao)

Bag of Words dataset de testes

In [750]:
testes = pd.read_excel(dataSet, sheet_name='TESTE_15%')
normaliza(testes)

testes_tweets = testes["text"].values
testes_classificacao = testes["rotulo"].values

vec_testes = vectorizer.transform(testes_tweets)
freq_testes= tfi.transform(vec_testes)

Biblioteca de métricas e cross-validation

In [751]:
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
import re

___

### **Algoritmos de Machine Learning**

*Para a criação dos modelos, utilizamos o método de cross validation, o qual realiza a validação cruzada do modelo. Neste caso, o modelo é dividido em 20 partes, treinado em 19 e testado em 1, assim não utilizando o dataset de validação*

#### *Naive Bayes:*

In [752]:
from sklearn.naive_bayes import MultinomialNB
#pd.set_option('display.max_rows', 110) #printa todas as linhas

modeloNB = MultinomialNB()
modeloNB.fit(freq_treino, treino_classificacao)

predicao = modeloNB.predict(freq_testes)

df_resultados = pd.DataFrame(data = { "Tweets": testes_tweets, "Rótulos obtidos": predicao, "Rótulos esperados": testes_classificacao })
df_resultados

,Tweets,Rótulos obtidos,Rótulos esperados
0,heinz progress prop mudanc transport publ regi...,POSITIVO,POSITIVO
1,@bandr obrig eduardo_leit invest tant saud #un...,POSITIVO,POSITIVO
2,@bandr fech eduardo_leit #uniaobrasilr #marcel...,POSITIVO,POSITIVO
3,edegar_prett deput vot part tre eleico partici...,POSITIVO,POSITIVO
4,@frenteampla13 unic candidat fal invest invest...,POSITIVO,POSITIVO
...,...,...,...
103,eduardo_leit porqu pag pis salar profes rio_gr...,NEGATIVO,NEGATIVO
104,porqu tant candidat era mont seguint chap edeg...,NEGATIVO,NEGATIVO
105,"""bolsoleite"" hoj ""denunciando"" falt invest fed...",NEGATIVO,NEGATIVO
106,escolh sim onyx escolh mort aind deix clar viv...,NEGATIVO,NEGATIVO


##### *Análise dos resultados obtidos com o NB*

In [753]:
resultados = cross_val_predict(modeloNB, freq_treino, treino_classificacao, cv = 10)
acrNB = metrics.accuracy_score(treino_classificacao, resultados)

print("Acurácia: ", "%.2f" % acrNB ,"%\n")
print(metrics.classification_report(treino_classificacao, resultados, target_names=['POSITIVO', 'NEGATIVO']))

Acurácia:  0.80 %

              precision    recall  f1-score   support

    POSITIVO       0.86      0.73      0.79       248
    NEGATIVO       0.76      0.88      0.82       248

    accuracy                           0.80       496
   macro avg       0.81      0.80      0.80       496
weighted avg       0.81      0.80      0.80       496



#### *Logistic Regression:*

In [754]:
from sklearn.linear_model import LogisticRegression
#pd.set_option('display.max_rows', 110) #printa todas as linhas

modeloLR = LogisticRegression(C = 2, max_iter = 1000, n_jobs=-1)
modeloLR.fit(freq_treino, treino_classificacao)

predicao = modeloLR.predict(freq_testes)

df_resultados = pd.DataFrame(data = { "Tweets": testes_tweets, "Rótulos obtidos": predicao, "Rótulos esperados": testes_classificacao })
df_resultados

,Tweets,Rótulos obtidos,Rótulos esperados
0,heinz progress prop mudanc transport publ regi...,NEGATIVO,POSITIVO
1,@bandr obrig eduardo_leit invest tant saud #un...,POSITIVO,POSITIVO
2,@bandr fech eduardo_leit #uniaobrasilr #marcel...,POSITIVO,POSITIVO
3,edegar_prett deput vot part tre eleico partici...,POSITIVO,POSITIVO
4,@frenteampla13 unic candidat fal invest invest...,POSITIVO,POSITIVO
...,...,...,...
103,eduardo_leit porqu pag pis salar profes rio_gr...,NEGATIVO,NEGATIVO
104,porqu tant candidat era mont seguint chap edeg...,NEGATIVO,NEGATIVO
105,"""bolsoleite"" hoj ""denunciando"" falt invest fed...",NEGATIVO,NEGATIVO
106,escolh sim onyx escolh mort aind deix clar viv...,NEGATIVO,NEGATIVO


##### *Análise dos resultados obtidos com o LP*

In [755]:
resultados = cross_val_predict(modeloLR, freq_treino, treino_classificacao, cv = 10)
acrLP = metrics.accuracy_score(treino_classificacao, resultados)

print("Acurácia: ", "%.2f" % acrLP ,"%\n")
print(metrics.classification_report(treino_classificacao, resultados, target_names=['POSITIVO', 'NEGATIVO']))

Acurácia:  0.85 %

              precision    recall  f1-score   support

    POSITIVO       0.82      0.89      0.85       248
    NEGATIVO       0.88      0.81      0.84       248

    accuracy                           0.85       496
   macro avg       0.85      0.85      0.85       496
weighted avg       0.85      0.85      0.85       496



#### *KNN:*

In [756]:
from sklearn.neighbors import KNeighborsClassifier

modeloKNN = KNeighborsClassifier(n_neighbors=9,algorithm='brute') #Using brute-force algorithm for quicker computation.
modeloKNN.fit(freq_treino, treino_classificacao)

predicao = modeloKNN.predict(freq_testes)

df_resultados = pd.DataFrame(data = { "Tweets": testes_tweets, "Rótulos obtidos": predicao, "Rótulos esperados": testes_classificacao })
df_resultados


,Tweets,Rótulos obtidos,Rótulos esperados
0,heinz progress prop mudanc transport publ regi...,POSITIVO,POSITIVO
1,@bandr obrig eduardo_leit invest tant saud #un...,POSITIVO,POSITIVO
2,@bandr fech eduardo_leit #uniaobrasilr #marcel...,POSITIVO,POSITIVO
3,edegar_prett deput vot part tre eleico partici...,POSITIVO,POSITIVO
4,@frenteampla13 unic candidat fal invest invest...,POSITIVO,POSITIVO
...,...,...,...
103,eduardo_leit porqu pag pis salar profes rio_gr...,POSITIVO,NEGATIVO
104,porqu tant candidat era mont seguint chap edeg...,NEGATIVO,NEGATIVO
105,"""bolsoleite"" hoj ""denunciando"" falt invest fed...",NEGATIVO,NEGATIVO
106,escolh sim onyx escolh mort aind deix clar viv...,NEGATIVO,NEGATIVO


##### *Análise dos resultados obtidos com o KNN*


In [757]:
resultados = cross_val_predict(modeloKNN, freq_treino, treino_classificacao, cv = 10)

acrKNN = metrics.accuracy_score(treino_classificacao, resultados)  

print("Acurácia: ", "%.2f" % acrKNN ,"%\n")
print(metrics.classification_report(treino_classificacao, resultados, target_names=['POSITIVO', 'NEGATIVO']))

Acurácia:  0.76 %

              precision    recall  f1-score   support

    POSITIVO       0.88      0.60      0.71       248
    NEGATIVO       0.70      0.92      0.79       248

    accuracy                           0.76       496
   macro avg       0.79      0.76      0.75       496
weighted avg       0.79      0.76      0.75       496



#### *MLP:*

In [758]:
from sklearn.neural_network import MLPClassifier

modeloMLP = MLPClassifier()
modeloMLP.fit(freq_treino, treino_classificacao)

predicao = modeloMLP.predict(freq_testes)

df_resultados = pd.DataFrame(data = { "Tweets": testes_tweets, "Rótulos obtidos": predicao, "Rótulos esperados": testes_classificacao })
df_resultados

,Tweets,Rótulos obtidos,Rótulos esperados
0,heinz progress prop mudanc transport publ regi...,NEGATIVO,POSITIVO
1,@bandr obrig eduardo_leit invest tant saud #un...,POSITIVO,POSITIVO
2,@bandr fech eduardo_leit #uniaobrasilr #marcel...,POSITIVO,POSITIVO
3,edegar_prett deput vot part tre eleico partici...,POSITIVO,POSITIVO
4,@frenteampla13 unic candidat fal invest invest...,POSITIVO,POSITIVO
...,...,...,...
103,eduardo_leit porqu pag pis salar profes rio_gr...,NEGATIVO,NEGATIVO
104,porqu tant candidat era mont seguint chap edeg...,NEGATIVO,NEGATIVO
105,"""bolsoleite"" hoj ""denunciando"" falt invest fed...",NEGATIVO,NEGATIVO
106,escolh sim onyx escolh mort aind deix clar viv...,NEGATIVO,NEGATIVO


##### *Análise dos resultados obtidos com o MLP*

In [759]:
#resultados = cross_val_predict(modeloMLP, freq_treino, treino_classificacao, cv = 10)
#acrMLP = metrics.accuracy_score(treino_classificacao, resultados)

#print("Acurácia: ", "%.2f" % acrMLP ,"%\n")
#print(metrics.classification_report(treino_classificacao, resultados, target_names=['POSITIVO', 'NEGATIVO']))

##**SLÁ MANINHO**

In [760]:
from collections import Counter

normaliza(treino)
treino['temp_list'] = treino['text'].apply(lambda x:str(x).split())
top = Counter([item for sublist in treino['temp_list'] for item in sublist])
temp = pd.DataFrame(top.most_common(100))
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Blues')

,Common_words,count
0,#debatenabandr,366
1,edegar_prett,171
2,eduardo_leit,145
3,govern,122
4,onyx,105
5,rio_grande_do_sul,88
6,#lulaolivioprettoru,85
7,candidat,57
8,debat,53
9,fal,53
